## 비교모델 학습

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import cv2
import timm  
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GroupShuffleSplit
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt 

# --- [시스템 설정] ---
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
cv2.setNumThreads(0) 
torch.backends.cudnn.benchmark = True # ⚡ 고속 학습 치트키

# ==========================================
# 1. 설정 및 경로 (다중 라벨 베이스라인 모드)
# ==========================================
CONFIG = {
    'base_dir': r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection',
    
    # 1. 3채널 파일 경로 (가짜 정답지 포함됨 ❌ -> 경로만 씁니다)
    'manifest_path': r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\det_train_manifest_2019_3ch.csv',
    
    # 2. 의사가 친 BBox 기반의 진짜 정답지 (사용자님 파일 ⭕ -> fracture 정답만 씁니다)
    'true_label_csv': r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset\det_train_manifest_cropped_full.csv',
    
    # 3. [추가] 주최측 원본 파일 (환자별 C1~C7 정답이 있는 파일)
    'train_csv_path': r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\train.csv',
    
    'model_name': 'tf_efficientnetv2_s.in21k_ft_in1k', 
    'img_size': 512,    
    'batch_size': 16,      
    'epochs': 100,          # 비교 모델이므로 30에폭 정도면 충분합니다.
    'patience': 10,       
    'learning_rate': 5e-5, 
    'weight_decay': 5e-2,  
    
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'save_dir': './runs/rsna_baseline_multilabel' # 📁 베이스라인 전용 폴더
}

# (데이터 증강은 이전과 동일하게 유지)
train_transforms = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.HorizontalFlip(p=0.5), 
    A.Affine(scale=(0.9, 1.1), translate_percent=(-0.05, 0.05), rotate=(-10, 10), p=0.5), 
    A.RandomBrightnessContrast(brightness_limit=0.15, contrast_limit=0.15, p=0.5), 
    A.GaussNoise(p=0.3),                                                                 
    A.CoarseDropout(max_holes=8, max_height=32, max_width=32, fill_value=0, p=0.3), 
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), 
    ToTensorV2()
])

val_transforms = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# ==========================================
# 2. 다중 라벨 전용 데이터셋
# ==========================================
class RSNAMultiLabelDataset(Dataset):
    def __init__(self, df, base_dir, transforms=None):
        self.df = df
        self.base_dir = base_dir
        self.transforms = transforms
        self.clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        
        full_path = os.path.join(self.base_dir, 'old_train', row['file_path'])
        try:
            with np.load(full_path) as loaded:
                img = loaded['data'].astype(np.float32) 
        except:
            img = np.zeros((3, 512, 512), dtype=np.float32)

        img_uint8 = (img * 255).astype(np.uint8)
        channels = [self.clahe.apply(img_uint8[i]) for i in range(3)]
        img_hwc = np.stack(channels, axis=-1) 
        
        if self.transforms:
            img_tensor = self.transforms(image=img_hwc)['image']
        else:
            img_tensor = torch.from_numpy(np.transpose(img_hwc, (2,0,1))).float() / 255.0

        # 🎯 [핵심] 7개의 정답을 하나의 배열로 묶어서 반환합니다! (C1, C2, C3, C4, C5, C6, C7)
        label = torch.tensor([
            float(row['target_C1']), float(row['target_C2']), float(row['target_C3']),
            float(row['target_C4']), float(row['target_C5']), float(row['target_C6']),
            float(row['target_C7'])
        ], dtype=torch.float32)
        
        return img_tensor, label

# ==========================================
# 3. 학습 및 검증 함수 (출력 7개)
# ==========================================
def train_one_epoch(model, dataloader, criterion, optimizer, device, scaler):
    model.train()
    running_loss = 0.0
    
    pbar = tqdm(dataloader, desc="Training", leave=False)
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        with torch.amp.autocast('cuda'): 
            outputs = model(images)  
            loss = criterion(outputs, labels) # BCEWithLogitsLoss가 7개 클래스 각각 계산
        
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item()
        pbar.set_postfix({'loss': f"{loss.item():.4f}"})
        
    return running_loss / len(dataloader)

@torch.no_grad()
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_preds = []
    all_targets = []
    
    pbar = tqdm(dataloader, desc="Validating", leave=False)
    for images, labels in pbar:
        images, labels = images.to(device), labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        
        probs = torch.sigmoid(outputs).cpu().numpy()
        targets = labels.cpu().numpy()
        
        all_preds.append(probs)
        all_targets.append(targets)
        
    all_preds = np.vstack(all_preds)   # Shape: (전체 슬라이스 수, 7)
    all_targets = np.vstack(all_targets) # Shape: (전체 슬라이스 수, 7)
    
    val_loss = running_loss / len(dataloader)
    
    # 7개 뼈 각각의 AUC를 구한 뒤 평균을 냅니다 (Macro AUC)
    auc_list = []
    for c in range(7):
        try:
            auc = roc_auc_score(all_targets[:, c], all_preds[:, c])
            auc_list.append(auc)
        except:
            pass # 해당 뼈에 골절 환자가 검증셋에 아예 없는 경우 에러 방지
            
    mean_auc = np.mean(auc_list) if len(auc_list) > 0 else 0.5
    
    return val_loss, mean_auc

# ==========================================
# 4. 메인 실행 (데이터 마법 조립)
# ==========================================
if __name__ == '__main__':
    os.makedirs(CONFIG['save_dir'], exist_ok=True)
    device = CONFIG['device']
    
    print("📖 데이터 로드 및 '7 다중 라벨' 결합 중...")
    
    df_3ch = pd.read_csv(CONFIG['manifest_path']) 
    df_true = pd.read_csv(CONFIG['true_label_csv'])
    df_train = pd.read_csv(CONFIG['train_csv_path']) # 주최측 원본 (환자별 C1~C7)
    
    df_3ch['StudyInstanceUID'] = df_3ch['StudyInstanceUID'].astype(str)
    df_true['StudyInstanceUID'] = df_true['StudyInstanceUID'].astype(str)
    df_train['StudyInstanceUID'] = df_train['StudyInstanceUID'].astype(str)
    
    # [1단계] 진짜 정답 + 3채널 경로 합체 (235명 정예)
    df_merged = pd.merge(
        df_3ch[['StudyInstanceUID', 'original_slice_number', 'file_path']], 
        df_true[['StudyInstanceUID', 'original_slice_number', 'fracture']], 
        on=['StudyInstanceUID', 'original_slice_number'], 
        how='inner'
    )
    
    # [2단계] 환자별 C1~C7 라벨 합체
    df_merged = pd.merge(
        df_merged,
        df_train[['StudyInstanceUID', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']],
        on='StudyInstanceUID',
        how='left'
    )
    
    # [3단계] 마법의 수식: 슬라이스 골절 여부(fracture)에 환자 정답(C1~C7)을 곱해줍니다!
    for c in range(1, 8):
        df_merged[f'target_C{c}'] = df_merged[f'C{c}'] * df_merged['fracture']
        
    df_235 = df_merged.reset_index(drop=True)
    print(f"✅ 다중 라벨(C1~C7) 정예 데이터 결합 완료! (총 {len(df_235):,}장)")

    # 🎯 환자 기준으로 Train(80%) / Val(20%) 분할
    gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=42)
    train_idx, val_idx = next(gss.split(df_235, groups=df_235['StudyInstanceUID']))
    
    train_df = df_235.iloc[train_idx].reset_index(drop=True)
    val_df = df_235.iloc[val_idx].reset_index(drop=True)
    
    # 🎯 Train 데이터 1:1 밸런싱
    train_frac = train_df[train_df['fracture'] == 1]
    train_norm = train_df[train_df['fracture'] == 0].sample(n=len(train_frac), random_state=42)
    train_df_balanced = pd.concat([train_frac, train_norm]).sample(frac=1).reset_index(drop=True)
    
    print("\n" + "="*40)
    print("⚖️ [데이터 분할 완료]")
    print(f"  - Train (1:1 강제) : {len(train_df_balanced):,}장")
    print(f"  - Val (전체 슬라이스): {len(val_df):,}장")
    print("="*40)
    
    train_ds = RSNAMultiLabelDataset(train_df_balanced, CONFIG['base_dir'], transforms=train_transforms)
    val_ds = RSNAMultiLabelDataset(val_df, CONFIG['base_dir'], transforms=val_transforms)
    
    train_loader = DataLoader(train_ds, batch_size=CONFIG['batch_size'], shuffle=True, num_workers=0, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=CONFIG['batch_size'], shuffle=False, num_workers=0, pin_memory=True)
    
    # 🛠️ 모델의 출력을 1 ➔ 7 로 변경 (num_classes=7)
    model = timm.create_model(CONFIG['model_name'], pretrained=True, in_chans=3, num_classes=7).to(device)
    
    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'], weight_decay=CONFIG['weight_decay'])
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG['epochs'])
    scaler = torch.amp.GradScaler('cuda') 
    
    best_auc = 0.0
    
    print(f"\n🚀 [Baseline 7-Class] 훈련 시작! (Model: {CONFIG['model_name']})")
    
    for epoch in range(1, CONFIG['epochs'] + 1):
        print(f"\n[Epoch {epoch}/{CONFIG['epochs']}]")
        
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device, scaler)
        val_loss, val_auc = validate(model, val_loader, criterion, device)
        scheduler.step() 
        
        print(f"📉 Train Loss: {train_loss:.4f} | 📈 Val Loss: {val_loss:.4f} | 🏆 Val Mean AUC: {val_auc:.4f}")
        
        if val_auc > best_auc:
            best_auc = val_auc
            save_path = os.path.join(CONFIG['save_dir'], "best_baseline_multilabel.pth")
            torch.save(model.state_dict(), save_path)
            print(f"💾 Best Model 저장됨! (Mean AUC: {best_auc:.4f}) 🔄")

## 학습 평가

In [ ]:
import os
import torch
import pandas as pd
import numpy as np
import cv2
import timm
import matplotlib.pyplot as plt
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import GroupShuffleSplit
import random

# --- [1. 설정] ---
CONFIG = {
    'base_dir': r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection',
    'manifest_path': r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\old_train\det_train_manifest_2019_3ch.csv',
    'true_label_csv': r'C:\conda\3. Project\rsna-2022-cervical-spine-fracture-detection\golden_dataset\det_train_manifest_cropped_full.csv',
    
    'model_name': 'tf_efficientnetv2_s.in21k_ft_in1k', 
    'img_size': 512,
    
    # 🎯 학습 완료된 베이스라인 가중치 경로
    'weights_path': './runs/rsna_baseline_multilabel/best_baseline_multilabel.pth', 
    
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'num_samples': 8
}

val_transforms = A.Compose([
    A.Resize(CONFIG['img_size'], CONFIG['img_size']),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# (데이터 로더는 위와 동일)
class RSNATestDataset(Dataset):
    def __init__(self, df, base_dir, transforms=None):
        self.df = df
        self.base_dir = base_dir
        self.transforms = transforms
        self.clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        
    def __len__(self): return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        full_path = os.path.join(self.base_dir, 'old_train', row['file_path'])
        try:
            with np.load(full_path) as loaded: img = loaded['data'].astype(np.float32) 
        except:
            img = np.zeros((3, 512, 512), dtype=np.float32)

        img_uint8 = (img * 255).astype(np.uint8)
        channels = [self.clahe.apply(img_uint8[i]) for i in range(3)]
        img_hwc = np.stack(channels, axis=-1) 
        
        if self.transforms: img_tensor = self.transforms(image=img_hwc)['image']
        else: img_tensor = torch.from_numpy(np.transpose(img_hwc, (2,0,1))).float() / 255.0

        label = float(row['fracture'])
        vis_img = channels[1] 
        return img_tensor, label, vis_img

def visualize_baseline_model():
    print("🛠️ 베이스라인 모델(7개 출력) 데이터 준비 중...")
    df_3ch = pd.read_csv(CONFIG['manifest_path']) 
    df_true = pd.read_csv(CONFIG['true_label_csv'])
    df_3ch['StudyInstanceUID'] = df_3ch['StudyInstanceUID'].astype(str)
    df_true['StudyInstanceUID'] = df_true['StudyInstanceUID'].astype(str)
    
    df_merged = pd.merge(
        df_3ch[['StudyInstanceUID', 'original_slice_number', 'file_path']], 
        df_true[['StudyInstanceUID', 'original_slice_number', 'fracture']], 
        on=['StudyInstanceUID', 'original_slice_number'], how='inner'
    ).reset_index(drop=True)

    gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=42)
    _, val_idx = next(gss.split(df_merged, groups=df_merged['StudyInstanceUID']))
    val_df = df_merged.iloc[val_idx].reset_index(drop=True)

    sample_df = val_df.sample(n=CONFIG['num_samples'], random_state=random.randint(0, 10000))
    test_loader = DataLoader(RSNATestDataset(sample_df, CONFIG['base_dir'], transforms=val_transforms), batch_size=CONFIG['num_samples'], shuffle=False)
    
    # 🚨 베이스라인 모델은 num_classes=7 입니다!
    model = timm.create_model(CONFIG['model_name'], pretrained=False, in_chans=3, num_classes=7)
    model.load_state_dict(torch.load(CONFIG['weights_path'], map_location=CONFIG['device']))
    model.to(CONFIG['device']).eval()
    
    with torch.no_grad():
        for images, labels, vis_imgs in test_loader:
            # 출력된 7개의 확률값을 받습니다.
            probs = torch.sigmoid(model(images.to(CONFIG['device']))).cpu().numpy()
            labels = labels.numpy()
            break 
            
    plt.figure(figsize=(16, 8))
    for i in range(CONFIG['num_samples']):
        plt.subplot(2, 4, i+1)
        plt.imshow(vis_imgs[i], cmap='gray')
        plt.axis('off')
        
        true_lbl = int(labels[i])
        
        # 7개 확률 중 가장 높은 확률(max)과 그 뼈의 인덱스(argmax)를 가져옵니다.
        max_prob = np.max(probs[i])
        max_idx = np.argmax(probs[i]) + 1 # 0인덱스가 C1이 되도록 +1
        
        pred_lbl = 1 if max_prob >= 0.5 else 0
        color = 'green' if true_lbl == pred_lbl else 'red'
        
        # 타이틀에 "몇 번 뼈(C)로 예측했는지"도 띄워줍니다!
        title_text = f"True: {true_lbl} | Pred: {max_prob:.2f} (C{max_idx})"
        plt.title(title_text, color=color, fontsize=13, fontweight='bold')
        
    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    visualize_baseline_model()